In [1]:
##aktuell beste version:

import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import tkinter as tk
from tkinter import ttk
import folium
import webbrowser
import tempfile
import pandas as pd
from sklearn.neighbors import BallTree

# API-Token für den Zugriff auf OpenRouteService
secrets = {
    'api_key': '5b3ce3597851110001cf624812d4898347ee4900b2b0e0eac8974a33'
}

# Pfad zur CSV-Datei mit den Ladesäulenstationen
csv_file_path = "/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/ladesauele_id.csv"

# Laden der Daten aus der CSV-Datei
data = pd.read_csv(csv_file_path)

# Funktion zur Berechnung der Entfernung zwischen zwei Koordinaten
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Funktion zur Berechnung der Gesamtdistanz
def calculate_total_distance(coordinates):
    total_distance = 0
    for i in range(1, len(coordinates)):
        coord1 = coordinates[i - 1]
        coord2 = coordinates[i]
        segment_distance = geodesic(coord1, coord2).kilometers
        total_distance += segment_distance
    return total_distance

# Funktion zur Geocodierung von Adressen
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Funktion zum Abrufen der Route und Anzeigen der Ergebnisse
def calculate_route():
    start_lat = float(entry_start_lat.get())
    start_long = float(entry_start_long.get())
    ziel_lat = float(entry_ziel_lat.get())
    ziel_long = float(entry_ziel_long.get())
    reichweite_km = float(entry_reichweite.get())

    # Karte erstellen
    m = folium.Map(location=[start_lat, start_long], zoom_start=10)

    # Marker für Start- und Ziel-Locations hinzufügen
    folium.Marker([start_lat, start_long], icon=folium.Icon(color='blue'), popup='Startpunkt').add_to(m)
    folium.Marker([ziel_lat, ziel_long], icon=folium.Icon(color='red'), popup='Zielpunkt').add_to(m)

    # Daten für den BallTree-Algorithmus vorbereiten
    coordinates = data[['Breitengrad', 'Längengrad']].values
    tree = BallTree(coordinates, metric=calculate_distance)

    # Anzahl der nächstgelegenen Nachbarn
    k_neighbors = 5

    # Finden der k-NN für Startpunkt und Zielpunkt
    start_neighbors = tree.query([[start_lat, start_long]], k=k_neighbors, return_distance=False)
    ziel_neighbors = tree.query([[ziel_lat, ziel_long]], k=k_neighbors, return_distance=False)

    # Ausgabe der 5 nächstgelegenen Ladesäulenstationen für Start- und Zielpunkt
    start_nearest_stations = [data.iloc[neighbor_idx]['ID'] for neighbor_idx in start_neighbors[0]]
    ziel_nearest_stations = [data.iloc[neighbor_idx]['ID'] for neighbor_idx in ziel_neighbors[0]]

    #Diese Prints können wir später gebrauchen
    #print("5 nächstgelegene Ladesäulenstationen vom Startpunkt:")
    #for station in start_nearest_stations:
        #print(station)
    #print("\n5 nächstgelegene Ladesäulenstationen vom Zielpunkt:")
    #for station in ziel_nearest_stations:
        #print(station)

    # Marker für Ladesäulenstationen hinzufügen
    for station_id in start_nearest_stations:
        station_data = data[data['ID'] == station_id]
        lat = station_data['Breitengrad'].values[0]
        lon = station_data['Längengrad'].values[0]
        station_name = station_data['Betreiber'].values[0]

        folium.Marker([lat, lon], icon=folium.Icon(color='green'), popup=station_name).add_to(m)

    for station_id in ziel_nearest_stations:
        station_data = data[data['ID'] == station_id]
        lat = station_data['Breitengrad'].values[0]
        lon = station_data['Längengrad'].values[0]
        station_name = station_data['Betreiber'].values[0]

        folium.Marker([lat, lon], icon=folium.Icon(color='green'), popup=station_name).add_to(m)

    # Route von OpenRouteService abrufen
    api_url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={secrets['api_key']}&start={start_long},{start_lat}&end={ziel_long},{ziel_lat}"
    response = requests.get(api_url)

    if response.status_code == 200:
        route_data = response.json()
        # Koordinaten der Route extrahieren
        route_coordinates = [(coord[1], coord[0]) for coord in route_data["features"][0]["geometry"]["coordinates"]]

        # Linienobjekt für die Route erstellen
        route_line = folium.PolyLine(route_coordinates, color="blue", weight=5, opacity=0.7)
        route_line.add_to(m)

        # Gesamtdistanz berechnen
        total_distance = calculate_total_distance(route_coordinates)

        # Karte als HTML speichern und in das Tkinter-UI einbetten
        tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
        m.save(tmpfile.name)

        webbrowser.open(f"file://{tmpfile.name}", new=1)

        # Ausgabe der Gesamtdistanz
        distance_label.config(text=f"Gesamtdistanz zwischen Start- und Zielpunkt: {total_distance:.2f} km")
    else:
        distance_label.config(text="Fehler beim Abrufen der Routendaten.")

# Tkinter-Fenster erstellen
root = tk.Tk()
root.title("Route Calculator")

# Eingabefelder für Startpunkt, Zielpunkt und Reichweite erstellen
frame = ttk.Frame(root)
frame.pack(padx=10, pady=10)

ttk.Label(frame, text="Startpunkt (Breitengrad):").grid(row=0, column=0, padx=5, pady=5)
ttk.Label(frame, text="Startpunkt (Längengrad):").grid(row=1, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zielpunkt (Breitengrad):").grid(row=2, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zielpunkt (Längengrad):").grid(row=3, column=0, padx=5, pady=5)
ttk.Label(frame, text="Reichweite (in km):").grid(row=4, column=0, padx=5, pady=5)

entry_start_lat = ttk.Entry(frame)
entry_start_lat.grid(row=0, column=1, padx=5, pady=5)
entry_start_long = ttk.Entry(frame)
entry_start_long.grid(row=1, column=1, padx=5, pady=5)
entry_ziel_lat = ttk.Entry(frame)  
entry_ziel_lat.grid(row=2, column=1, padx=5, pady=5)
entry_ziel_long = ttk.Entry(frame)
entry_ziel_long.grid(row=3, column=1, padx=5, pady=5)
entry_reichweite = ttk.Entry(frame)
entry_reichweite.grid(row=4, column=1, padx=5, pady=5)

# Schaltfläche zum Berechnen der Route erstellen
calculate_button = ttk.Button(frame, text="Route berechnen", command=calculate_route)
calculate_button.grid(row=5, columnspan=2, padx=5, pady=10)

# Label für die Ausgabe der Gesamtdistanz erstellen
distance_label = ttk.Label(frame, text="")
distance_label.grid(row=6, columnspan=2, padx=5, pady=10)

root.mainloop()


In [4]:
## Unfertige Version für Adresseingabe anstatt Längenbreiten Grad

import requests
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import tkinter as tk
from tkinter import ttk
import folium
import webbrowser
import tempfile
import pandas as pd
from sklearn.neighbors import BallTree

# API-Token für den Zugriff auf OpenRouteService
secrets = {
    'api_key': '5b3ce3597851110001cf624812d4898347ee4900b2b0e0eac8974a33'
}

# Pfad zur CSV-Datei mit den Ladesäulenstationen
csv_file_path = "/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/ladesauele_id.csv"

# Laden der Daten aus der CSV-Datei
data = pd.read_csv(csv_file_path)


# Funktion zur Berechnung der Entfernung zwischen zwei Koordinaten
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Funktion zur Berechnung der Gesamtdistanz
def calculate_total_distance(coordinates):
    total_distance = 0
    for i in range(1, len(coordinates)):
        coord1 = coordinates[i - 1]
        coord2 = coordinates[i]
        segment_distance = geodesic(coord1, coord2).kilometers
        total_distance += segment_distance
    return total_distance

# Funktion zur Geocodierung von Adressen
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address, ssl_verify=False)  # Deaktivieren der SSL-Verifikation
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Initialisieren der Variablen für die nächstgelegenen Ladesäulenstationen
start_nearest_stations = []
ziel_nearest_stations = []

# Funktion zum Abrufen der Route und Anzeigen der Ergebnisse
def calculate_route():
    # Initialisieren der Variablen für die nächstgelegenen Ladesäulenstationen
    start_nearest_stations = []
    ziel_nearest_stations = []

    start_address = entry_start_address.get()
    ziel_address = entry_ziel_address.get()
    reichweite_km = float(entry_reichweite.get())

    # Geocodierung der Start- und Zieladressen
    start_location = geocode_address(start_address)
    ziel_location = geocode_address(ziel_address)

    if start_location is None or ziel_location is None:
        distance_label.config(text="Fehler bei der Adressgeocodierung.")
        return

    start_lat, start_long = start_location
    ziel_lat, ziel_long = ziel_location

    # Karte erstellen
    m = folium.Map(location=[start_lat, start_long], zoom_start=10)

    # Marker für Start- und Ziel-Locations hinzufügen
    folium.Marker([start_lat, start_long], icon=folium.Icon(color='blue'), popup='Startpunkt').add_to(m)
    folium.Marker([ziel_lat, ziel_long], icon=folium.Icon(color='red'), popup='Zielpunkt').add_to(m)

    # Restlicher Code für die Routenberechnung bleibt unverändert
    # ...

# Tkinter-Fenster erstellen
root = tk.Tk()
root.title("Route Calculator")

# Eingabefelder für Start- und Zieladresse sowie Reichweite erstellen
frame = ttk.Frame(root)
frame.pack(padx=10, pady=10)

ttk.Label(frame, text="Startadresse:").grid(row=0, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zieladresse:").grid(row=1, column=0, padx=5, pady=5)
ttk.Label(frame, text="Reichweite (in km):").grid(row=2, column=0, padx=5, pady=5)

entry_start_address = ttk.Entry(frame)
entry_start_address.grid(row=0, column=1, padx=5, pady=5)
entry_ziel_address = ttk.Entry(frame)
entry_ziel_address.grid(row=1, column=1, padx=5, pady=5)
entry_reichweite = ttk.Entry(frame)
entry_reichweite.grid(row=2, column=1, padx=5, pady=5)

# Schaltfläche zum Berechnen der Route erstellen
calculate_button = ttk.Button(frame, text="Route berechnen", command=calculate_route)
calculate_button.grid(row=3, columnspan=2, padx=5, pady=10)

# Label für die Ausgabe der Gesamtdistanz erstellen
distance_label = ttk.Label(frame, text="")
distance_label.grid(row=4, columnspan=2, padx=5, pady=10)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/d5/dqk535g15sl7j06k2gd0hyc40000gn/T/ipykernel_77421/3768102602.py", line 64, in calculate_route
    start_location = geocode_address(start_address)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/d5/dqk535g15sl7j06k2gd0hyc40000gn/T/ipykernel_77421/3768102602.py", line 43, in geocode_address
    location = geolocator.geocode(address, ssl_verify=False)  # Deaktivieren der SSL-Verifikation
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Nominatim.geocode() got an unexpected keyword argument 'ssl_verify'


In [3]:
# Test Version für die Implementierung von zwischen Stops in der Mitte der Route zwischen Start und Zielpunkt:


import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import tkinter as tk
from tkinter import ttk
import folium
import webbrowser
import tempfile
import pandas as pd
from sklearn.neighbors import BallTree

# API-Token für den Zugriff auf OpenRouteService
secrets = {
    'api_key': '5b3ce3597851110001cf624812d4898347ee4900b2b0e0eac8974a33'
}

# Pfad zur CSV-Datei mit den Ladesäulenstationen
csv_file_path = "/Users/takomon/Documents/IT/TechLabs/team2/team02/datasets/2.landing/ladesauele_id.csv"

# Laden der Daten aus der CSV-Datei
data = pd.read_csv(csv_file_path)

# Funktion zur Berechnung der Entfernung zwischen zwei Koordinaten
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Funktion zur Berechnung der Gesamtdistanz
def calculate_total_distance(coordinates):
    total_distance = 0
    for i in range(1, len(coordinates)):
        coord1 = coordinates[i - 1]
        coord2 = coordinates[i]
        segment_distance = geodesic(coord1, coord2).kilometers
        total_distance += segment_distance
    return total_distance

# Funktion zur Geocodierung von Adressen
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Funktion zum Abrufen der Route und Anzeigen der Ergebnisse
def calculate_route():
    start_lat = float(entry_start_lat.get())
    start_long = float(entry_start_long.get())
    ziel_lat = float(entry_ziel_lat.get())
    ziel_long = float(entry_ziel_long.get())
    reichweite_km = float(entry_reichweite.get())

    # Karte erstellen
    m = folium.Map(location=[start_lat, start_long], zoom_start=10)

    # Marker für Start- und Ziel-Locations hinzufügen
    folium.Marker([start_lat, start_long], icon=folium.Icon(color='blue'), popup='Startpunkt').add_to(m)
    folium.Marker([ziel_lat, ziel_long], icon=folium.Icon(color='red'), popup='Zielpunkt').add_to(m)

    # Daten für den BallTree-Algorithmus vorbereiten
    coordinates = data[['Breitengrad', 'Längengrad']].values
    tree = BallTree(coordinates, metric=calculate_distance)

    # Anzahl der nächstgelegenen Nachbarn
    k_neighbors = 5

    # Finden der k-NN für Startpunkt und Zielpunkt
    start_neighbors = tree.query([[start_lat, start_long]], k=k_neighbors, return_distance=False)
    ziel_neighbors = tree.query([[ziel_lat, ziel_long]], k=k_neighbors, return_distance=False)

    # Ausgabe der 5 nächstgelegenen Ladesäulenstationen für Start- und Zielpunkt
    start_nearest_stations = [data.iloc[neighbor_idx]['ID'] for neighbor_idx in start_neighbors[0]]
    ziel_nearest_stations = [data.iloc[neighbor_idx]['ID'] for neighbor_idx in ziel_neighbors[0]]


    # Marker für Ladesäulenstationen hinzufügen
    for station_id in start_nearest_stations:
        station_data = data[data['ID'] == station_id]
        lat = station_data['Breitengrad'].values[0]
        lon = station_data['Längengrad'].values[0]
        station_name = station_data['Betreiber'].values[0]

        folium.Marker([lat, lon], icon=folium.Icon(color='green'), popup=station_name).add_to(m)

    for station_id in ziel_nearest_stations:
        station_data = data[data['ID'] == station_id]
        lat = station_data['Breitengrad'].values[0]
        lon = station_data['Längengrad'].values[0]
        station_name = station_data['Betreiber'].values[0]

        folium.Marker([lat, lon], icon=folium.Icon(color='green'), popup=station_name).add_to(m)

    # Route von OpenRouteService abrufen
    api_url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={secrets['api_key']}&start={start_long},{start_lat}&end={ziel_long},{ziel_lat}"
    response = requests.get(api_url)

    if response.status_code == 200:
        route_data = response.json()
        # Koordinaten der Route extrahieren
        route_coordinates = [(coord[1], coord[0]) for coord in route_data["features"][0]["geometry"]["coordinates"]]

        # Linienobjekt für die Route erstellen
        route_line = folium.PolyLine(route_coordinates, color="blue", weight=5, opacity=0.7)
        route_line.add_to(m)

        # Gesamtdistanz berechnen
        total_distance = calculate_total_distance(route_coordinates)

        # Karte als HTML speichern und in das Tkinter-UI einbetten
        tmpfile = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
        m.save(tmpfile.name)

        webbrowser.open(f"file://{tmpfile.name}", new=1)

        # Ausgabe der Gesamtdistanz
        distance_label.config(text=f"Gesamtdistanz zwischen Start- und Zielpunkt: {total_distance:.2f} km")

        # Hinzufügen von Ladestationen in der Mitte der Strecke
        add_charging_stations_on_route(m, route_coordinates, num_stations=5)

    else:
        distance_label.config(text="Fehler beim Abrufen der Routendaten.")

# Funktion zum Hinzufügen von Ladestationen in der Mitte der Strecke
def add_charging_stations_on_route(map_obj, route_coordinates, num_stations=5):
    if len(route_coordinates) < 2:
        return
    
    # Finden des Mittelpunkts der Route
    mid_idx = len(route_coordinates) // 2
    mid_coord = route_coordinates[mid_idx]
    
    # Finden der nächsten Ladestationen zum Mittelpunkt
    nearest_stations = find_nearest_stations(mid_coord[0], mid_coord[1], num_stations)
    
    # Hinzufügen von Markierungen für die Ladestationen
    for station in nearest_stations:
        lat = station['Breitengrad']
        lon = station['Längengrad']
        station_name = station['Betreiber']
        folium.Marker([lat, lon], icon=folium.Icon(color='purple'), popup=station_name).add_to(map_obj)

# Funktion zum Finden der nächsten Ladestationen zu einer Koordinate
def find_nearest_stations(lat, lon, num_stations):
    station_data = data[['Breitengrad', 'Längengrad', 'Betreiber']].copy()
    station_data['distance'] = station_data.apply(lambda row: geodesic((lat, lon), (row['Breitengrad'], row['Längengrad'])).kilometers, axis=1)
    nearest_stations = station_data.nsmallest(num_stations, 'distance')
    return nearest_stations.to_dict(orient='records')

# Tkinter-Fenster erstellen
root = tk.Tk()
root.title("Route Calculator")

# Eingabefelder für Startpunkt, Zielpunkt und Reichweite erstellen
frame = ttk.Frame(root)
frame.pack(padx=10, pady=10)

ttk.Label(frame, text="Startpunkt (Breitengrad):").grid(row=0, column=0, padx=5, pady=5)
ttk.Label(frame, text="Startpunkt (Längengrad):").grid(row=1, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zielpunkt (Breitengrad):").grid(row=2, column=0, padx=5, pady=5)
ttk.Label(frame, text="Zielpunkt (Längengrad):").grid(row=3, column=0, padx=5, pady=5)
ttk.Label(frame, text="Reichweite (in km):").grid(row=4, column=0, padx=5, pady=5)

entry_start_lat = ttk.Entry(frame)
entry_start_lat.grid(row=0, column=1, padx=5, pady=5)
entry_start_long = ttk.Entry(frame)
entry_start_long.grid(row=1, column=1, padx=5, pady=5)
entry_ziel_lat = ttk.Entry(frame)  
entry_ziel_lat.grid(row=2, column=1, padx=5, pady=5)
entry_ziel_long = ttk.Entry(frame)
entry_ziel_long.grid(row=3, column=1, padx=5, pady=5)
entry_reichweite = ttk.Entry(frame)
entry_reichweite.grid(row=4, column=1, padx=5, pady=5)

# Schaltfläche zum Berechnen der Route erstellen
calculate_button = ttk.Button(frame, text="Route berechnen", command=calculate_route)
calculate_button.grid(row=5, columnspan=2, padx=5, pady=10)

# Label für die Ausgabe der Gesamtdistanz erstellen
distance_label = ttk.Label(frame, text="")
distance_label.grid(row=6, columnspan=2, padx=5, pady=10)

root.mainloop()